In [2]:
## Imports

# utility modules
import glob
import os
import sys
import re

# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

In [3]:
# user inputs the ground track number and beam you want to find results for
rgt = "0531"
beam = 'gt1l'        #options:  gt1l, gt1r, gt2l, gt2r, gt3l, gt3r

# folder path where tracks are stored, change this according to whether you are looking in the Greenland/Antarctica folder
data_root = '/srv/shared/FirnAndMelt/ATL03/Antarctica/'

In [4]:
# list to store the data we want to plot
plot_data = []

# iterate over each repeat cycle
for cycle in ['01','02','03','04','05','06','07']:

    try:
        # read the IS2 data with Tyler's ATL03 reader:
        ATL03_file=glob.glob(os.path.join(data_root, f'*{rgt}{cycle}*.h5'))[0]
        IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)

        # add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
        get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

        #-- select the 2l beam from ATL03
        D3 = IS2_atl03_mds[beam]
        LMH=D3['heights']['signal_conf_ph'][:,3] >= 2

        # add the data we will want to plot
        #[x_atc all photons, h all photons, x_atc flagged, h flagged, cycle, date]
        d = [ D3['heights']['x_atc'], D3['heights']['h_ph'], D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH], cycle, ATL03_file[-33:-29]+"-"+ATL03_file[-29:-27]+"-"+ATL03_file[-27:-25] ]
        plot_data.append(d)
        print("Cycle " + cycle + " added")
    
    except Exception as e:
        print(f'cycle = {cycle}, exception = {e}')
        print()

Cycle 01 added
Cycle 02 added
Cycle 03 added
Cycle 04 added
Cycle 05 added
Cycle 06 added
cycle = 07, exception = list index out of range



In [20]:
# create a scatter plot for each cycle
plt.close()

# these values change the locations you are looking at along the track
x_min = 32071500
x_max = 32073500
y_min = 102
y_max = 112

fig, axes = plt.subplots(nrows=len(plot_data)//2, ncols=2, figsize=(10, 10), sharex=True, sharey=True)

for c, ax in zip(plot_data, axes.flatten()):
    ax.plot(c[0],c[1],'k.',markersize=0.25, label='Date: '+str(c[5]) + '  Cycle: ' + str(c[4]))
    #ax.plot(c[2],c[3],'g.',markersize=0.5)
    ax.legend(loc='upper right',)
    #ax.set_ylabel('Height (m)')
    ax.set_xlim((x_min,x_max))
    ax.set_ylim((y_min,y_max))
    
fig.suptitle('Ground Track: ' + rgt, y=1)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …